<a href="https://colab.research.google.com/github/Shivay-Shakti/Crowdfunding-Analytics/blob/main/V6_Boosted_RFM_Features_Month_Removed_Table_Max_Rendition_BQ_Hackathon_StarterNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# Author : Lavi Nigam, ML Engineering @ Google 
# Linkedin: https://www.linkedin.com/in/lavinigam/ 

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/lavinigam-gcp/BQML_Hackathon/blob/main/LTV_Prediction/BQ_Hackathon_StarterNotebook.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/lavinigam-gcp/BQML_Hackathon/blob/main/LTV_Prediction/BQ_Hackathon_StarterNotebook.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
</table>

Before starting the notebook, make sure you have two things: 

1) Acknowledging that you finished the pre-requisite [here](https://machinehack.com/hackathons/google_cloud_bigquery_ltv_prediction_challenge/overview?prerequisites=true)

2) Project ID from Google Cloud. 

Do not proceed further wihout these steps. 

Set your project ID here.

In [ ]:
PROJECT_ID = ""
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "mh-bq-hackathon-375416"  # @param {type:"string"}

PROJECT_ID

'mh-bq-hackathon-375416'

### Authenticate your Google Cloud account

***Ensure you are logged out of all google (Gmail) accounts except the one you created for this Hackathon. We recommend using a different browser or making sure you have opened Colab using Hackathon's newly created Gmail account..***

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

In [ ]:
# Importing some important libraries that will be used during the notebook
import pandas as pd
import plotly.express as px
from google.cloud import bigquery

In [ ]:
#Client manages connections to the BigQuery API and helps
#bundle configuration (project, credentials) needed for API requests.
client = bigquery.Client(PROJECT_ID)

# to make sure all columns are displayed while working with dataframe
pd.set_option('display.max_columns', None)

## Assumptions

## Exploratory Data Analysis (EDA)

You can start by defining some essential variables that can change according to your data. It is always better to consider the most recent records from your data as features. For this purpose, you can set the START_DATE and END_DATE based on your data recency.

In this case, the date range is set for 3 months.

In [ ]:
PROJECT_ID_DATA = "mh-hackathon"
DATASET_ID_DATA = "ga4_data" 
TABLE_ID_TRAIN  = "ga4_train"
TABLE_ID_TEST  = "ga4_test"
START_DATE = "20201101"
END_DATE = "20210131"

You can start the data exploration by returning the first five rows of data.
The data has multiple event tables for each day. So, all the tables (events) could be queried by using events* as the wildcard.

[GA4 Data Export Schema](https://support.google.com/analytics/answer/7029846#zippy=)

Note: BigQuery export, by default, are [date sharded tables](https://cloud.google.com/bigquery/docs/partitioned-tables#dt_partition_shard)

In [ ]:
query = f"""
SELECT
  *
FROM
  `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.{TABLE_ID_TRAIN}`
LIMIT
  5
"""
print(query)
query_job = client.query(query)
top5_data = query_job.to_dataframe()
top5_data.head()


SELECT
  *
FROM
  `mh-hackathon.ga4_data.ga4_train`
LIMIT
  5



,user_pseudo_id,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,privacy_info,user_properties,user_first_touch_timestamp,device,geo,app_info,traffic_source,stream_id,platform,event_dimensions,ecommerce,items,ltv
0,8594243.4562718268,20201121,1605972615267100,view_item,"[{'key': 'page_location', 'value': {'string_va...",<NA>,NaN,782266748,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1605971991634862,"{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Asia', 'sub_continent': 'Wester...",None,"{'medium': '(none)', 'name': '(direct)', 'sour...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[],70.052761
1,30900021.3233786964,20210108,1610080403881070,user_engagement,"[{'key': 'debug_mode', 'value': {'string_value...",<NA>,NaN,7456154266,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1602033447181464,"{'category': 'mobile', 'mobile_brand_name': '<...","{'continent': 'Asia', 'sub_continent': 'Wester...",None,"{'medium': '(none)', 'name': '(direct)', 'sour...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[],8.012165
2,2661490.8294165879,20201110,1605002723977128,page_view,"[{'key': 'page_title', 'value': {'string_value...",<NA>,NaN,316922447,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1605002723977128,"{'category': 'mobile', 'mobile_brand_name': '<...","{'continent': 'Asia', 'sub_continent': 'Wester...",None,"{'medium': '<Other>', 'name': '<Other>', 'sour...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[],70.900515
3,52407464.0313773688,20201208,1607461234617356,scroll,"[{'key': 'session_engaged', 'value': {'string_...",<NA>,NaN,-7325297035,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1605035923000488,"{'category': 'mobile', 'mobile_brand_name': 'H...","{'continent': 'Asia', 'sub_continent': 'Wester...",None,"{'medium': 'referral', 'name': '(referral)', '...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[],0.779221
4,84123100.3290360033,20201108,1604864504346804,user_engagement,"[{'key': 'debug_mode', 'value': {'string_value...",<NA>,NaN,199154306,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1604864373827450,"{'category': 'mobile', 'mobile_brand_name': 'A...","{'continent': 'Asia', 'sub_continent': 'Wester...",None,"{'medium': 'referral', 'name': '(referral)', '...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[],8.388783


In [ ]:
query = f"""
SELECT
  *
FROM
  `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.{TABLE_ID_TEST}`
LIMIT
  5
"""
print(query)
query_job = client.query(query)
top5_data = query_job.to_dataframe()
top5_data.head()


SELECT
  *
FROM
  `mh-hackathon.ga4_data.ga4_test`
LIMIT
  5



,user_pseudo_id,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,privacy_info,user_properties,user_first_touch_timestamp,device,geo,app_info,traffic_source,stream_id,platform,event_dimensions,ecommerce,items
0,1000823.8498711409,20210118,1610938668079160,page_view,"[{'key': 'session_engaged', 'value': {'string_...",<NA>,NaN,-9796371683,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1610938663062961,"{'category': 'mobile', 'mobile_brand_name': 'A...","{'continent': 'Americas', 'sub_continent': 'So...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]
1,1000823.8498711409,20210118,1610938740311812,user_engagement,"[{'key': 'page_title', 'value': {'string_value...",<NA>,NaN,-2370677642,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1610938663062961,"{'category': 'mobile', 'mobile_brand_name': 'A...","{'continent': 'Americas', 'sub_continent': 'So...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]
2,1000823.8498711409,20210118,1610938663062961,session_start,"[{'key': 'ga_session_number', 'value': {'strin...",<NA>,NaN,9869243676,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1610938663062961,"{'category': 'mobile', 'mobile_brand_name': 'A...","{'continent': 'Americas', 'sub_continent': 'So...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]
3,1000985.4712566084,20210114,1610603132099178,session_start,"[{'key': 'page_title', 'value': {'string_value...",<NA>,NaN,2236704977,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1610603132099178,"{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Americas', 'sub_continent': 'No...",None,"{'medium': '(none)', 'name': '(direct)', 'sour...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]
4,1000985.4712566084,20210114,1610603132099178,page_view,"[{'key': 'engaged_session_event', 'value': {'s...",<NA>,NaN,2236704977,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1610603132099178,"{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Americas', 'sub_continent': 'No...",None,"{'medium': '(none)', 'name': '(direct)', 'sour...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]


The first five rows of data can help you understand the tables' composite structure of data types. For example, you can see numerical, categorical, Arrays, and Struct as data types. Using this information, later, you will be able to write specific `UNNEST` queries for [Arrays](https://cloud.google.com/bigquery/docs/reference/standard-sql/arrays#query_structs_in_an_array) & [Struct](https://cloud.google.com/bigquery/docs/reference/standard-sql/arrays#querying_array-type_fields_in_a_struct).

By looking at some columns, you can also identify a few essential features like event_date, event_name, user_ltv, device, geo, traffic_source, platform, and items. However, as discussed earlier, you still are not aware of their value distribution, availability, and data types.

You can check the data types of each column using [INFORMATION_SCHEMA](https://cloud.google.com/bigquery/docs/information-schema-tables) table. It can give you detailed metadata of your columns.

In [ ]:
query = f"""
SELECT
  DISTINCT(column_name),
  data_type
FROM
  `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.INFORMATION_SCHEMA.COLUMNS`
"""
print(query)
query_job = client.query(query)
predict_data = query_job.to_dataframe()
predict_data


SELECT
  DISTINCT(column_name),
  data_type
FROM
  `mh-hackathon.ga4_data.INFORMATION_SCHEMA.COLUMNS`



,column_name,data_type
0,user_pseudo_id,STRING
1,event_date,STRING
2,event_timestamp,INT64
3,event_name,STRING
4,event_params,"ARRAY<STRUCT<key STRING, value STRUCT<string_v..."
5,event_previous_timestamp,INT64
6,event_value_in_usd,FLOAT64
7,event_bundle_sequence_id,INT64
8,event_server_timestamp_offset,INT64
9,user_id,STRING


In [ ]:
query = f"""
select * from `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.{TABLE_ID_TRAIN}`, UNNEST (event_params) as ep limit 1000
"""
print(query)
query_job = client.query(query)
predict_data = query_job.to_dataframe()
predict_data


select * from `mh-hackathon.ga4_data.ga4_train`, UNNEST (event_params) as ep limit 1000



,user_pseudo_id,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,privacy_info,user_properties,user_first_touch_timestamp,device,geo,app_info,traffic_source,stream_id,platform,event_dimensions,ecommerce,items,ltv,key,value
0,8594243.4562718268,20201121,1605972615267100,view_item,"[{'key': 'page_location', 'value': {'string_va...",<NA>,NaN,782266748,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1605971991634862,"{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Asia', 'sub_continent': 'Wester...",None,"{'medium': '(none)', 'name': '(direct)', 'sour...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[],70.052761,page_location,{'string_value': 'https://shop.googlemerchandi...
1,8594243.4562718268,20201121,1605972615267100,view_item,"[{'key': 'page_location', 'value': {'string_va...",<NA>,NaN,782266748,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1605971991634862,"{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Asia', 'sub_continent': 'Wester...",None,"{'medium': '(none)', 'name': '(direct)', 'sour...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[],70.052761,ga_session_id,"{'string_value': None, 'int_value': 712494394,..."
2,8594243.4562718268,20201121,1605972615267100,view_item,"[{'key': 'page_location', 'value': {'string_va...",<NA>,NaN,782266748,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1605971991634862,"{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Asia', 'sub_continent': 'Wester...",None,"{'medium': '(none)', 'name': '(direct)', 'sour...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[],70.052761,engagement_time_msec,"{'string_value': None, 'int_value': 43, 'float..."
3,8594243.4562718268,20201121,1605972615267100,view_item,"[{'key': 'page_location', 'value': {'string_va...",<NA>,NaN,782266748,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1605971991634862,"{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Asia', 'sub_continent': 'Wester...",None,"{'medium': '(none)', 'name': '(direct)', 'sour...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[],70.052761,clean_event,"{'string_value': 'gtm.js', 'int_value': None, ..."
4,8594243.4562718268,20201121,1605972615267100,view_item,"[{'key': 'page_location', 'value': {'string_va...",<NA>,NaN,782266748,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1605971991634862,"{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Asia', 'sub_continent': 'Wester...",None,"{'medium': '(none)', 'name': '(direct)', 'sour...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[],70.052761,session_engaged,"{'string_value': '1', 'int_value': None, 'floa..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,49972222.2824139250,20201113,1605306837450772,page_view,"[{'key': 'debug_mode', 'value': {'string_value...",<NA>,NaN,6045229697,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1605305993473565,"{'category': 'tablet', 'mobile_brand_name': 'H...","{'continent': 'Americas', 'sub_continent': 'No...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[],11.798957,ga_session_number,"{'string_value': None, 'int_value': 1, 'float_..."
996,49972222.2824139250,20201113,1605306837450772,page_view,"[{'key': 'debug_mode', 'value': {'string_value...",<NA>,NaN,6045229697,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1605305993473565,"{'category': 'tablet', 'mobile_brand_name': 'H...","{'continent': 'Americas', 'sub_continent': 'No...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': 

You can start by understanding overall data by getting a quick summary of the data, namely - total events  (event_count), total users (user_count), total days in the data (day_count), and total registered users of the platform (registered_user_id).
This can help you get a sense of the scale of data.

In [ ]:
query = f"""

SELECT
  COUNT(*) AS event_count,
  COUNT(DISTINCT user_pseudo_id) AS user_count,
  COUNT(DISTINCT event_date) AS day_count,
  COUNT(DISTINCT user_id) AS registered_user_id
FROM
  `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.{TABLE_ID_TRAIN}`
"""
print(query)
query_job = client.query(query)
top5_data = query_job.to_dataframe()
top5_data



SELECT
  COUNT(*) AS event_count,
  COUNT(DISTINCT user_pseudo_id) AS user_count,
  COUNT(DISTINCT event_date) AS day_count,
  COUNT(DISTINCT user_id) AS registered_user_id
FROM
  `mh-hackathon.ga4_data.ga4_train`



,event_count,user_count,day_count,registered_user_id
0,3859763,243394,92,0


In [ ]:
query = f"""

SELECT
  COUNT(*) AS event_count,
  COUNT(DISTINCT user_pseudo_id) AS user_count,
  COUNT(DISTINCT event_date) AS day_count,
  COUNT(DISTINCT user_id) AS registered_user_id
FROM
  `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.{TABLE_ID_TEST}`
"""
print(query)
query_job = client.query(query)
top5_data = query_job.to_dataframe()
top5_data



SELECT
  COUNT(*) AS event_count,
  COUNT(DISTINCT user_pseudo_id) AS user_count,
  COUNT(DISTINCT event_date) AS day_count,
  COUNT(DISTINCT user_id) AS registered_user_id
FROM
  `mh-hackathon.ga4_data.ga4_test`



,event_count,user_count,day_count,registered_user_id
0,435821,26760,92,0


In [ ]:
top5_data.head()

,event_count,user_count,day_count,registered_user_id
0,435821,26760,92,0


As you can observe, there are roughly 4 million events with close to 270,000 users, stretched along 92 days of activity on the platform.

There are no registered users data in the table. The user_pseudo_id is not a "user_id"; it is an client ID (cookie ID) for the user. This means that a single user can be represented as multiple pseudo_id in the data.

For simplicity, we will assume that all user_pseudo_id are unique and represent a single user.

If your data has 'user_id', use that directly, or else you can go ahead and use 'user_psuudo_id'.


### How to access Nested Data



---
Now, you can start by looking into `event_name` distribution.

event_name is a significant column in this dataset. It contains all the events triggered as users interact with the Google Merchandise Store like page_view, scroll (scrolling the page), view_item (viewing specific item), etc. You can refer [here](https://developers.google.com/analytics/devguides/collection/ga4/reference/events) for a more detailed meaning of each event_name.


In [ ]:
query = f"""
SELECT
  event_name,
  COUNT(*) as row_count
FROM
   `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.{TABLE_ID_TRAIN}`
GROUP BY
  event_name
ORDER BY
  row_count DESC
"""
print(query)
query_job = client.query(query)
result_df = query_job.to_dataframe()
fig = px.bar(result_df, x="row_count", y="event_name",  title="Event Name Frequency Distribution")
fig.show()


SELECT
  event_name,
  COUNT(*) as row_count
FROM
   `mh-hackathon.ga4_data.ga4_train`
GROUP BY
  event_name
ORDER BY
  row_count DESC



can you do this for each of the event_name strut?

#### Accessing Array Struct

In [ ]:
query = f"""
SELECT
  DISTINCT(ep.key) AS event_param_key,
  COUNT(*) AS count
FROM
  `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.{TABLE_ID_TRAIN}`,
  UNNEST (event_params) AS ep
WHERE
  event_name = 'page_view'
GROUP BY
  ep.key
ORDER BY
  count DESC
"""
print(query)
query_job = client.query(query)
result_df = query_job.to_dataframe()
result_df


SELECT
  DISTINCT(ep.key) AS event_param_key,
  COUNT(*) AS count
FROM
  `mh-hackathon.ga4_data.ga4_train`,
  UNNEST (event_params) AS ep
WHERE
  event_name = 'page_view'
GROUP BY
  ep.key
ORDER BY
  count DESC



,event_param_key,count
0,ga_session_number,1213143
1,ga_session_id,1213143
2,session_engaged,1213143
3,page_location,1213143
4,debug_mode,1213077
5,page_title,1207539
6,engaged_session_event,1174367
7,all_data,1012876
8,clean_event,1012803
9,page_referrer,891451


In [ ]:
query = f"""
SELECT
  event_date,
  event_timestamp,
  event_name,
  user_pseudo_id,
  ep.key,
  ep.value
FROM
  `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.{TABLE_ID_TRAIN}`,
  UNNEST (event_params) AS ep
WHERE
  ep.key = 'page_title'
LIMIT
  10
  """
print(query)
query_job = client.query(query)
result_df = query_job.to_dataframe()
result_df.head()


SELECT
  event_date,
  event_timestamp,
  event_name,
  user_pseudo_id,
  ep.key,
  ep.value
FROM
  `mh-hackathon.ga4_data.ga4_train`,
  UNNEST (event_params) AS ep
WHERE
  ep.key = 'page_title'
LIMIT
  10
  


,event_date,event_timestamp,event_name,user_pseudo_id,key,value
0,20201203,1607033908954509,page_view,52407464.0313773688,page_title,{'string_value': 'New | Google Merchandise Sto...
1,20201203,1607032477375233,session_start,52407464.0313773688,page_title,"{'string_value': 'Home', 'int_value': None, 'f..."
2,20201229,1609202428116997,user_engagement,84760672.8640407660,page_title,{'string_value': 'Bags | Lifestyle | Google Me...
3,20201208,1607391347326167,first_visit,29934948.8545379469,page_title,"{'string_value': 'Google Online Store', 'int_v..."
4,20201229,1609268001007966,session_start,8370439.6464221452,page_title,{'string_value': 'YouTube | Shop by Brand | Go...


In [ ]:
query = f"""
SELECT
  event_date,
  event_timestamp,
  event_name,
  user_pseudo_id,
  ep.key,
  ep.value.string_value
FROM
  `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.{TABLE_ID_TRAIN}`,
  UNNEST (event_params) AS ep
WHERE
  ep.key = 'page_title'
LIMIT
  10
  """
print(query)
query_job = client.query(query)
result_df = query_job.to_dataframe()
result_df.head()


SELECT
  event_date,
  event_timestamp,
  event_name,
  user_pseudo_id,
  ep.key,
  ep.value.string_value
FROM
  `mh-hackathon.ga4_data.ga4_train`,
  UNNEST (event_params) AS ep
WHERE
  ep.key = 'page_title'
LIMIT
  10
  


,event_date,event_timestamp,event_name,user_pseudo_id,key,string_value
0,20201121,1605972615267100,view_item,8594243.4562718268,page_title,Google Speckled Beanie Navy
1,20210108,1610080403881070,user_engagement,30900021.3233786964,page_title,Men's / Unisex | Apparel | Google Merchandise ...
2,20201110,1605002723977128,page_view,2661490.8294165879,page_title,Men's T-Shirts | Apparel | Google Merchandise ...
3,20201208,1607461234617356,scroll,52407464.0313773688,page_title,The Google Merchandise Store - Log In
4,20201108,1604864504346804,user_engagement,84123100.3290360033,page_title,Shopping Cart


#### Accessing Struct

In [ ]:
#I have commented it out
query = f"""
SELECT
  event_date,
  event_timestamp,
  event_name,
  user_pseudo_id,
  geo.continent,
  geo.sub_continent,
  geo.country,
  geo.region
FROM
  `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.{TABLE_ID_TRAIN}`
LIMIT
  10
  """
print(query)
query_job = client.query(query)
result_df = query_job.to_dataframe()
#result_df.head()


SELECT
  event_date,
  event_timestamp,
  event_name,
  user_pseudo_id,
  geo.continent,
  geo.sub_continent,
  geo.country,
  geo.region
FROM
  `mh-hackathon.ga4_data.ga4_train`
LIMIT
  10
  


## Feature Engineering

Before we create our features, its important to create a dataset, where our new feature table will be stored. 

In [ ]:
# You can create the dataset through code.
DATASET_NAME = "ga4_ecomm_feature_set"

try:
  dataset = client.create_dataset(DATASET_NAME, timeout=30)  # Make an API request.
  print("Created dataset {}.{}".format(client.project, dataset.dataset_id))
except Exception as e:
  print(e)

409 POST https://bigquery.googleapis.com/bigquery/v2/projects/mh-bq-hackathon-375416/datasets?prettyPrint=false: Already Exists: Dataset mh-bq-hackathon-375416:ga4_ecomm_feature_set


Lets create some features that we can levrage to create Kmeans Cluster. For this example, we will simply take browsing data of each user. 

In [63]:
feature_table="ga4_features_train" #table name

In [64]:
query = f"""
CREATE OR REPLACE TABLE
  {DATASET_NAME}.{feature_table} AS
SELECT
user_pseudo_id, AVG(ltv) as avg_ltv,
MAX(IFNULL((SELECT value.int_value FROM UNNEST(event_params) WHERE key = 'ga_session_number'), 0)) AS ga_session_number,
Max(device.operating_system) as operating_system,
MAX(device.language) as language,
SUM(ecommerce.total_item_quantity) as item_quantity,
SUM(ecommerce.tax_value_in_usd) as tax_value_usd,
SUM(ecommerce.purchase_revenue_in_usd) as revenue_usd,
MAX(traffic_source.medium) as medium,
COUNT(user_pseudo_id) as frequency,
MAX(event_date) as recency,
MAX(event_timestamp-user_first_touch_timestamp) as duration,
SUM(value.int_value) AS value_int, 
MAX(geo.country) AS country,
MAX(CAST(format_date('%d',parse_date("%Y%m%d",event_date)) as INT64)) as day_of_the_month,
MAX(CAST(format_date('%w',parse_date("%Y%m%d",event_date)) as INT64)) as day_of_week,

FROM 
`{PROJECT_ID_DATA}.{DATASET_ID_DATA}.{TABLE_ID_TRAIN}`,UNNEST (event_params)
group by user_pseudo_id
  """
#print(query)
query_job = client.query(query)

In [65]:
#table to see what have we placed in the features train table 

query = """

SELECT
  *
FROM
  `ga4_ecomm_feature_set.ga4_features_train`
LIMIT
5
"""
print(query)
query_job = client.query(query)
result_df = query_job.to_dataframe()
result_df.head()



SELECT
  *
FROM
  `ga4_ecomm_feature_set.ga4_features_train`
LIMIT
5



,user_pseudo_id,avg_ltv,ga_session_sum,operating_system,language,item_quantity,tax_value_usd,revenue_usd,medium,frequency,recency,duration,value_int,country,day_of_the_month,day_of_week
0,4399255.4686925870,296.680894,91,iOS,en-us,<NA>,NaN,NaN,organic,91,20201101,21774323,32068393129,India,1,0
1,1832198.0226996350,29.995080,81,Web,en-us,<NA>,NaN,NaN,referral,81,20201101,35012794,38129438101,United States,1,0
2,85321426.0561924417,70.839964,50,Web,en-us,<NA>,NaN,NaN,(none),36,20201101,45135981798,16731124089,Mexico,1,0
3,3986275.8951856120,77.489139,124,Web,zh,<NA>,NaN,NaN,organic,76,20201101,4429072637,14637671800,United States,1,0
4,84283047.0221023972,58.119044,97,Web,en-us,<NA>,NaN,NaN,(none),97,20201101,413462408,20713763473,Greece,1,0


In [73]:
model_name = "ltv_boosted_tree"

boosted_tree_query = f"""
CREATE OR REPLACE MODEL
  `{DATASET_NAME}.{model_name}` 
  OPTIONS (
      MODEL_TYPE =  'BOOSTED_TREE_REGRESSOR',
      LEARN_RATE = 0.08,
      BOOSTER_TYPE = 'GBTREE',
      COLSAMPLE_BYTREE = 0.6,
      SUBSAMPLE = 0.8,
      MAX_TREE_DEPTH = 7,
      INPUT_LABEL_COLS = ['avg_ltv']
   ) 
AS
SELECT
  * except (user_pseudo_id)
FROM
  `{PROJECT_ID}.{DATASET_NAME}.{feature_table}`
WHERE
  avg_ltv IS NOT NULL
"""
#print(boosted_tree_query)
query_job = client.query(boosted_tree_query)






In [74]:
ml_evaluate_query = f"""
SELECT
  *
FROM
  ML.EVALUATE(MODEL `{DATASET_NAME}.{model_name}`,
    (
    SELECT
      *
    FROM
      `{PROJECT_ID}.{DATASET_NAME}.{feature_table}`
    WHERE
      avg_ltv IS NOT NULL))
"""
print(ml_evaluate_query)
query_job = client.query(ml_evaluate_query)
ml_info_df = query_job.to_dataframe()
ml_info_df




SELECT
  *
FROM
  ML.EVALUATE(MODEL `ga4_ecomm_feature_set.ltv_boosted_tree`,
    (
    SELECT
      *
    FROM
      `mh-bq-hackathon-375416.ga4_ecomm_feature_set.ga4_features_train`
    WHERE
      avg_ltv IS NOT NULL))



,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,68.527561,11400.560644,2.590214,49.922926,-0.053964,-0.05301


## Prediction on Test & Submission

In [ ]:
# Creating features from test set, added value int, geo country and there is no ltv because its prediction
#in test we are supposed to remove avg_ltv the thing that we are predicting
feature_table_test ="ga4_features_test" #table name
query = f"""
CREATE OR REPLACE TABLE
  {DATASET_NAME}.{feature_table_test} AS
SELECT
user_pseudo_id,
MAX(IFNULL((SELECT value.int_value FROM UNNEST(event_params) WHERE key = 'ga_session_number'), 0)) AS ga_session_number,
Max(device.operating_system) as operating_system,
MAX(device.language) as language,
SUM(ecommerce.total_item_quantity) as item_quantity,
SUM(ecommerce.tax_value_in_usd) as tax_value_usd,
SUM(ecommerce.purchase_revenue_in_usd) as revenue_usd,
MAX(traffic_source.medium) as medium, 
COUNT(user_pseudo_id) as frequency,
MAX(event_date) as recency,
MAX(event_timestamp-user_first_touch_timestamp) as duration,
SUM(value.int_value) AS value_int, 
MAX(geo.country) AS country,
MAX(CAST(format_date('%d',parse_date("%Y%m%d",event_date)) as INT64)) as day_of_the_month,
MAX(CAST(format_date('%w',parse_date("%Y%m%d",event_date)) as INT64)) as day_of_week,

FROM 
`{PROJECT_ID_DATA}.{DATASET_ID_DATA}.{TABLE_ID_TEST}`,UNNEST (event_params)
group by user_pseudo_id
  """
#print(query) #had to do this otherwise the code did not execute
query_job = client.query(query)

In [ ]:
query = """

SELECT
  *
FROM
  `ga4_ecomm_feature_set.ga4_features_test`
LIMIT
5
"""

query_job = client.query(query)
result_df = query_job.to_dataframe()
result_df.head()

,user_pseudo_id,frequency,recency,duration,value_int,country,month_of_the_year,week_of_the_year,day_of_the_month,day_of_week
0,9273656154.0812702791,22,20201101,0,13822227741,Egypt,11,44,1,0
1,5625121.4675055110,92,20201101,99326019,27112139873,Chile,11,44,1,0
2,4484133.3083872151,321,20201101,14503129562,189302338810,Israel,11,44,1,0
3,2260071.3619448689,56,20201101,9902487,3217818491,Slovakia,11,44,1,0
4,63991504.2404816880,97,20201101,21474515,26432127265,Vietnam,11,44,1,0


In [ ]:
prediction_data_table_name = "model_prediction_ltv_test"
query = f"""
CREATE OR REPLACE TABLE
  {DATASET_NAME}.{prediction_data_table_name} AS
SELECT
  *
FROM
  ML.PREDICT(MODEL `{DATASET_NAME}.{model_name}`,
    (
    SELECT
      *
    FROM
      `{PROJECT_ID}.{DATASET_NAME}.ga4_features_test`
    ))
"""
#print(query)
query_job = client.query(query)

In [ ]:
query = f"""
SELECT
  *
FROM
  {PROJECT_ID}.{DATASET_NAME}.{prediction_data_table_name}
LIMIT 5
"""
print(query)
query_job = client.query(query)
predict_data = query_job.to_dataframe()
predict_data.head()


SELECT
  *
FROM
  mh-bq-hackathon-375416.ga4_ecomm_feature_set.model_prediction_ltv_test
LIMIT 5



,trial_id,predicted_avg_ltv,user_pseudo_id,frequency,recency,duration,value_int,country,month_of_the_year,week_of_the_year,day_of_the_month,day_of_week
0,1,102.995171,63991504.2404816880,97,20201101,21474515,26432127265,Vietnam,11,44,1,0
1,1,100.434311,17069994.6979099687,22,20201101,0,25500528915,Lithuania,11,44,1,0
2,1,97.592308,9448025.7094294807,138,20201101,144651956,46996840503,Israel,11,44,1,0
3,1,104.751801,1064366401.3806029006,33,20201101,4499810,30733597240,Czechia,11,44,1,0
4,1,103.607483,27113295.7125812899,107,20201101,90926251,76650688459,Malaysia,11,44,1,0


In [ ]:
query = f"""
SELECT
  user_pseudo_id,predicted_avg_ltv
FROM
  {PROJECT_ID}.{DATASET_NAME}.{prediction_data_table_name}
"""
# print(query)
query_job = client.query(query)
submission_data = query_job.to_dataframe()
submission_data.head()

,user_pseudo_id,predicted_avg_ltv
0,63991504.2404816880,102.995171
1,17069994.6979099687,100.434311
2,9448025.7094294807,97.592308
3,1064366401.3806029006,104.751801
4,27113295.7125812899,103.607483


In [ ]:
## Create Submission 
submission = submission_data[['user_pseudo_id',"predicted_avg_ltv"]]
submission.to_csv("submission.csv",index=False)
submission.head()

,user_pseudo_id,predicted_avg_ltv
0,63991504.2404816880,102.995171
1,17069994.6979099687,100.434311
2,9448025.7094294807,97.592308
3,1064366401.3806029006,104.751801
4,27113295.7125812899,103.607483


In [ ]:
#make sure before submission that your shape is (26760, 2). Do not submit unless its of same shape. 
submission.shape

(26760, 2)